In [69]:

# The goal for today is to automate the collection of weather data for multiple cities. 

1# Explore the data received from the API. What’s useful? What’s not?
2# Extract the information you see as useful and make a DataFrame from it
3#  the steps you took to make the DataFrame into a function. The function should output a DataFrame with the weather of multiple cities, when given a list of cities as an input. If you’re still unsure about functions, here’s a guide.
4# You should be able to use this function to get the weather data for the cities you web scraped yesterday

#STEP 1: Make a single API CALL AND EXPLORE THE DATA

import requests
import pandas as pd

API_KEY = "da22f5405712ba228e3f2b844d0d4326"

def get_weather_raw(city):
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    r = requests.get(url)
    return r.json()

    


In [71]:
# explore the returned JSON
data = get_weather_raw("Berlin")
data.keys()

# list(contains the forecast every 3 hrs) # city(city name , country, coordinates)

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [86]:
{
   'dt_txt': '2024-06-05 12:00:00',
   'main': {'temp': 15.4, 'humidity': 80},
   'weather': [{'description': 'light rain'}],
   'wind': {'speed': 4.0}
}


{'dt_txt': '2024-06-05 12:00:00',
 'main': {'temp': 15.4, 'humidity': 80},
 'weather': [{'description': 'light rain'}],
 'wind': {'speed': 40}}

In [84]:
# STEP 2: Extract the useful pices
1 #city name
2 # datetime
3 # temperature
4 # humditity
5 # weather description
6 # wind speed

def extract_weather(data):
    city_name = data['city']['name']
    records = []

    for entry in data['list']:
        records.append({
            "city": city_name,
            "datetime": entry['dt_txt'],
            "temp": entry['main']['temp'],
            "humidity": entry['main']['humidity'],
            "weather": entry['weather'][0]['description'],
            "wind_speed": entry['wind']['speed']
        })
    
    return pd.DataFrame(records)


In [73]:
df = extract_weather(data)
df.head()


,city,datetime,temp,humidity,weather,wind_speed
0,Berlin,2025-11-17 15:00:00,5.32,82,light rain,3.70
1,Berlin,2025-11-17 18:00:00,4.68,81,broken clouds,4.20
2,Berlin,2025-11-17 21:00:00,3.25,80,few clouds,3.35
3,Berlin,2025-11-18 00:00:00,1.62,85,clear sky,3.99
4,Berlin,2025-11-18 03:00:00,-0.07,84,few clouds,4.71


In [ ]:
#STEP 3: combine everything one function


In [74]:
["Berlin", "Hamburg", "Paris"]


['Berlin', 'Hamburg', 'Paris']

In [85]:
def get_weather_multiple(cities):
    all_dfs = []

    for city in cities:
        data = get_weather_raw(city)
        
        # Skip failed cities
        if data.get("cod") != "200":
            print(f"Could not fetch data for {city}")
            continue

        df_city = extract_weather(data)
        all_dfs.append(df_city)
    
    return pd.concat(all_dfs, ignore_index=True)


In [87]:
#final step - run the full automation

cities = ["Berlin", "Hamburg", "Munich", "Cologne"]
df_all = get_weather_multiple(cities)
df_all.head()

,city,datetime,temp,humidity,weather,wind_speed
0,Berlin,2025-11-17 18:00:00,3.49,84,broken clouds,4.20
1,Berlin,2025-11-17 21:00:00,2.68,81,scattered clouds,3.35
2,Berlin,2025-11-18 00:00:00,1.62,85,clear sky,3.99
3,Berlin,2025-11-18 03:00:00,-0.07,84,few clouds,4.71
4,Berlin,2025-11-18 06:00:00,3.22,83,broken clouds,4.98
